**Objective of the notebook:**

In this notebook, let us explore the given dataset and make some inferences along the way. Also finally we will build a baseline light gbm model to get started. 

**Objective of the competition:**

In this competition, we a’re challenged to analyze a Google Merchandise Store (also known as GStore, where Google swag is sold) customer dataset to predict revenue per customer. 

In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import datetime
from fastai.structured import *
from fastai.column_data import *

%matplotlib inline

#from sklearn import model_selection, preprocessing, metrics
#import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

**About the dataset:**

Similar to most other kaggle competitions, we are given two datasets
* train.csv
* test.csv

Each row in the dataset is one visit to the store. We are predicting the natural log of the sum of all transactions per user. 
    
The data fields in the given files are 
* fullVisitorId- A unique identifier for each user of the Google Merchandise Store.
* channelGrouping - The channel via which the user came to the Store.
* date - The date on which the user visited the Store.
* device - The specifications for the device used to access the Store.
* geoNetwork - This section contains information about the geography of the user.
* sessionId - A unique identifier for this visit to the store.
* socialEngagementType - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
* totals - This section contains aggregate values across the session.
* trafficSource - This section contains information about the Traffic Source from which the session originated.
* visitId - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
* visitNumber - The session number for this user. If this is the first session, then this is set to 1.
* visitStartTime - The timestamp (expressed as POSIX time).

Also it is important to note that some of the fields are in json format. 

Thanks to this [wonderful kernel](https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields/notebook) by [Julian](https://www.kaggle.com/julian3833), we can convert all the json fields in the file to a flattened csv format which generally use in other competitions.

In [2]:
location = 'work'
if(location == 'work'):
    PATH = "C:\\Users\\SCatheline\\Documents\\Kaggle Data\\Google Analytics\\"
elif(location == 'home'):
    PATH = ""
else:
    print('Unknown location.')

In [3]:
df_train = pd.read_csv('train_preprocessed.csv')

C:\Users\SCatheline\AppData\Local\Continuum\anaconda3\envs\tensorflow-GPU\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_train=df_train.drop(columns = df_train.columns[0])

In [5]:
df_test = pd.read_csv('test_preprocessed.csv')

C:\Users\SCatheline\AppData\Local\Continuum\anaconda3\envs\tensorflow-GPU\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df_test=df_test.drop(columns = df_test.columns[0])

Now let us create development and validation splits based on time to build the model. We can take the last two months as validation sample.

In [7]:
# Impute 0 for missing target values
df_train["totals.transactionRevenue"].fillna(0, inplace=True)
train_y = df_train["totals.transactionRevenue"].values
train_id = df_train["fullVisitorId"].values
test_id = df_test["fullVisitorId"].values

In [8]:
cont_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']

In [9]:
for col in cont_cols:
    df_train[col] = df_train[col].astype(float)
    df_test[col] = df_test[col].astype(float)

In [10]:
#Find the earliest and latest date
print(min(df_train['date']))
print(max(df_train['date']))

20160801
20170801


### Create new columns based on the date

In [11]:
df_train['Weekday'] = np.zeros((df_train.shape[0],),dtype = int)
df_train['Month'] = np.zeros((df_train.shape[0],),dtype = int)
df_train['Day'] = np.zeros((df_train.shape[0],),dtype = int)
df_train['date'] = df_train['date'].astype('str')

In [12]:
for i in range(df_train.shape[0]):
    current_date = datetime.datetime.strptime(df_train['date'].values[i],'%Y%m%d').date()
    df_train['Weekday'].values[i] = current_date.weekday()
    df_train['Month'].values[i] = current_date.month
    df_train['Day'].values[i] = current_date.day
    df_train['date'].values[i] = current_date

In [13]:
df_train.head(3)

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,Weekday,Month,Day
0,4,2016-09-02,1131660440785968503,1472830385,1.0,1.472830e+09,35,0,False,20,378,3,210,0,37454,193,21,1.0,1.0,1.0,1.0,0,62,3,59008,1,11,3,4,1,11,5,3196,208,4,9,2
1,4,2016-09-02,377306020877927890,1472880147,1.0,1.472880e+09,43,0,False,7,955,5,12,122,10098,482,1,1.0,1.0,1.0,1.0,0,62,3,59008,1,11,3,4,1,11,5,3196,208,4,9,2
2,4,2016-09-02,3895546263509774583,1472865386,1.0,1.472865e+09,35,0,False,20,475,4,185,0,38725,99,19,1.0,1.0,1.0,1.0,0,62,3,59008,1,11,3,4,1,11,5,3196,208,4,9,2


In [14]:
df_test['Weekday'] = np.zeros((df_test.shape[0],),dtype = int)
df_test['Month'] = np.zeros((df_test.shape[0],),dtype = int)
df_test['Day'] = np.zeros((df_test.shape[0],),dtype = int)
df_test['date'] = df_test['date'].astype('str')

In [15]:
for i in range(df_test.shape[0]):
    current_date = datetime.datetime.strptime(df_test['date'].values[i],'%Y%m%d').date()
    df_test['Weekday'].values[i] = current_date.weekday()
    df_test['Month'].values[i] = current_date.month
    df_test['Day'].values[i] = current_date.day
    df_test['date'].values[i] = current_date

In [16]:
df_train.head(3)

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,Weekday,Month,Day
0,4,2016-09-02,1131660440785968503,1472830385,1.0,1.472830e+09,35,0,False,20,378,3,210,0,37454,193,21,1.0,1.0,1.0,1.0,0,62,3,59008,1,11,3,4,1,11,5,3196,208,4,9,2
1,4,2016-09-02,377306020877927890,1472880147,1.0,1.472880e+09,43,0,False,7,955,5,12,122,10098,482,1,1.0,1.0,1.0,1.0,0,62,3,59008,1,11,3,4,1,11,5,3196,208,4,9,2
2,4,2016-09-02,3895546263509774583,1472865386,1.0,1.472865e+09,35,0,False,20,475,4,185,0,38725,99,19,1.0,1.0,1.0,1.0,0,62,3,59008,1,11,3,4,1,11,5,3196,208,4,9,2


In [17]:
val_idx = df_train.index[df_train['date']>datetime.date(2017,5,31)]

In [18]:
val_idx

Int64Index([  4822,   4823,   4824,   4825,   4826,   4827,   4828,   4829,
              4830,   4831,
            ...
            901253, 901254, 901255, 901256, 901257, 901258, 901259, 901260,
            901261, 901262],
           dtype='int64', length=137946)

In [19]:
df_train = df_train.set_index("date")
df_test = df_test.set_index("date")

In [20]:
# label encode the categorical variables and convert the numerical variables to float
cat_cols = ["Weekday", "Month", "Day", "channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

In [21]:
for v in cat_cols: df_train[v] = df_train[v].astype('category').cat.as_ordered()

In [22]:
apply_cats(df_test, df_train)

In [23]:
# Split the train dataset into development and valid based on time 
#dev_df = df_train[df_train['date']<=datetime.date(2017,5,31)]
#val_df = df_train[df_train['date']datetime.date(2017,5,31)]
#dev_y = np.log1p(dev_df["totals.transactionRevenue"].values)
#val_y = np.log1p(val_df["totals.transactionRevenue"].values)

df_training = df_train[cat_cols + cont_cols]
y = np.log(df_train["totals.transactionRevenue"].values)

#dev_X = dev_df[cat_cols + cont_cols] 
#val_X = val_df[cat_cols + cont_cols] 
test_X = df_test[cat_cols + cont_cols] 

C:\Users\SCatheline\AppData\Local\Continuum\anaconda3\envs\tensorflow-GPU\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In [24]:
max_log_y = max(np.log(y))
y_range = (0, max_log_y*1.2)

C:\Users\SCatheline\AppData\Local\Continuum\anaconda3\envs\tensorflow-GPU\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


In [25]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df_train, y, cat_flds=cat_cols, bs=2,
                                       test_df=test_X)

In [26]:
cat_sz = [(c, len(df_train[c].cat.categories)+1) for c in cat_cols]

In [27]:
cat_sz

[('Weekday', 8),
 ('Month', 13),
 ('Day', 32),
 ('channelGrouping', 9),
 ('device.browser', 55),
 ('device.deviceCategory', 4),
 ('device.operatingSystem', 21),
 ('geoNetwork.city', 650),
 ('geoNetwork.continent', 7),
 ('geoNetwork.country', 223),
 ('geoNetwork.metro', 95),
 ('geoNetwork.networkDomain', 28065),
 ('geoNetwork.region', 377),
 ('geoNetwork.subContinent', 24),
 ('trafficSource.adContent', 46),
 ('trafficSource.adwordsClickInfo.adNetworkType', 4),
 ('trafficSource.adwordsClickInfo.gclId', 17776),
 ('trafficSource.adwordsClickInfo.page', 10),
 ('trafficSource.adwordsClickInfo.slot', 4),
 ('trafficSource.campaign', 11),
 ('trafficSource.keyword', 3661),
 ('trafficSource.medium', 8),
 ('trafficSource.referralPath', 1477),
 ('trafficSource.source', 381),
 ('trafficSource.adwordsClickInfo.isVideoAd', 3),
 ('trafficSource.isTrueDirect', 3)]

In [28]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [29]:
emb_szs

[(8, 4),
 (13, 7),
 (32, 16),
 (9, 5),
 (55, 28),
 (4, 2),
 (21, 11),
 (650, 50),
 (7, 4),
 (223, 50),
 (95, 48),
 (28065, 50),
 (377, 50),
 (24, 12),
 (46, 23),
 (4, 2),
 (17776, 50),
 (10, 5),
 (4, 2),
 (11, 6),
 (3661, 50),
 (8, 4),
 (1477, 50),
 (381, 50),
 (3, 2),
 (3, 2)]

In [30]:
m = md.get_learner(emb_szs, len(df_train.columns)-len(cat_cols),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
m.summary()

C:\Users\SCatheline\AppData\Local\Continuum\anaconda3\envs\tensorflow-GPU\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


OrderedDict([('Embedding-1',
              OrderedDict([('input_shape', [-1]),
                           ('output_shape', [-1, 4]),
                           ('trainable', True),
                           ('nb_params', tensor(32))])),
             ('Embedding-2',
              OrderedDict([('input_shape', [-1]),
                           ('output_shape', [-1, 7]),
                           ('trainable', True),
                           ('nb_params', tensor(91))])),
             ('Embedding-3',
              OrderedDict([('input_shape', [-1]),
                           ('output_shape', [-1, 16]),
                           ('trainable', True),
                           ('nb_params', tensor(512))])),
             ('Embedding-4',
              OrderedDict([('input_shape', [-1]),
                           ('output_shape', [-1, 5]),
                           ('trainable', True),
                           ('nb_params', tensor(45))])),
             ('Embedding-5',
              Ord

In [31]:
lr = 1e-3
m.lr_find()

  0%|          | 0/382854 [00:00<?, ?it/s]

C:\Users\SCatheline\AppData\Local\Continuum\anaconda3\envs\tensorflow-GPU\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


RuntimeError: cuda runtime error (4) : unspecified launch failure at C:/ProgramData/Miniconda3/conda-bld/pytorch_1533090623466/work/aten/src/THC/THCTensorCopy.cu:206

In [ ]:
m.sched.plot(100)

In [ ]:
lr = 1e-3

In [ ]:
m.fit(lr, 3, metrics=[mse])

In [ ]:
m.save('val0')

In [ ]:
m.load('val0')

In [ ]:
x,y=m.predict_with_targs()

In [ ]:
pred_test=m.predict(True)

In [ ]:
pred_test = np.exp(pred_test)

In [ ]:
#joined_test['Sales']=pred_test